# EnKF equation

We will now analyze how Data Assimilation (DA) through the EnKF combines model and observation information. See lecture "Introduction to Data Assimilation" for more information on the EnKF equation. 

## Setting up the DA run

The DA run is built upon an ensemble run, as the ensemble information will be used to compute the uncertainty of the model. **Please copy-paste the missing lines from Exercise 2.**

In [ ]:
import numpy as np

# Define parameter K
############ TO BE FILLED ##################

# Define timestep
############ TO BE FILLED ##################

# precipitation data
############ TO BE FILLED ##################

# initialize storage vector
############ TO BE FILLED ##################

# Define ensemble size
############ TO BE FILLED ##################

# Load gaussian noise files
############ TO BE FILLED ##################

# Set noise amplitude
############ TO BE FILLED ##################

# Perturb variables
############ TO BE FILLED ##################


We will now continue setting up the Data Assimilation run. First, we need to get some observations. In this case, as this bucket model is a ficticious scenario, we will generate synthetic water storage observations. For that purpose, we start by generating a synthetic ground truth:

In [ ]:
from functions import linearModel

# %% Generate synthetic ground truth
sim = np.array([0, 23])  # Python is 0-indexed
S0_true = 8
S_true = np.zeros(sim[1] - sim[0] + 1)
S_true[0] = S0_true
Q_true, S_true = linearModel.linearModel(S_true, P[sim[0]:sim[1] + 1], K, Dt)


We will now generate the observations, which represent a noisy measurement of this synthetic truth. **[Marker for extra exercises].**

In [ ]:
# %% Generate observations
noise_S_obs = np.load('data/noise_S_obs.npy')
sigma_sObs = 0.3
offset = 0
S_obs = S_true + offset + S_true * sigma_sObs * noise_S_obs.T[0]

Now that the observations are generated, we will perturb them to then assimilate them throught the EnKF.

In [ ]:
# Perturb observations
Y = np.tile(S_obs[:, np.newaxis], (1, Ne))
noise_dY = np.load('data/noise_dY.npy')
dY = np.tile(S_true[:, np.newaxis], (1, Ne)) * sigma_sObs * noise_dY
Y_ens = Y + dY
Y_ens[Y_ens < 0] = 0

Another variable we need for the DA run is the design matrix, which relates the model state to the observed variable. In this case, the relation is 1-to-1.

In [ ]:
# %% Design matrix
A = np.array([[1]])

## Initialization run

We will now intialize the model over a period of 12 timesteps. **Can you copy-paste the missing lines from Exercise 2?**

In [ ]:
# %% Initialization run
############ TO BE FILLED ##################


## Open Loop run (Ensemble run)

We will also run the ensemble run as done in the previous exercise, so that we can plot it and copare it with the DA results. In the field of land DA, this is often referred to as "Open Loop (OL)" run. **Please copy-paste the missing lines from Exercise 2.**

In [ ]:
# %% Open Loop run
############ TO BE FILLED ##################


## Data Assimilation run

We finally launch the DA run. First, let us initialize the variables that we will be using.

In [ ]:
# % Initialization
S_t = S0_ens.copy()
n_steps = sim[1] - sim[0] + 1

xMinus_3d = np.zeros((1, Ne, n_steps))
xPlus_3d = np.zeros_like(xMinus_3d)
Cxx_3d = np.zeros((1, 1, n_steps))
Cxpxp_3d = np.zeros_like(Cxx_3d)
Sll_2d = np.zeros((n_steps, 1))

During DA, we will iterate over the timesteps combining two steps:

In [ ]:
# % loop over time
for t in range(sim[0], sim[1] + 1):
#   % Step 1: forward model run
#   % in this step, we will run the model forward of one timestep for all of the 
#   % ensemble members:
    Q_tp1 = np.zeros((1, Ne))
    S_tp1 = np.zeros((1, Ne))
    for ii in range(Ne):
        Q_tp1[0, ii], S_tp1[0, ii] = linearModel.linearModel(
            np.array([S_t[0,ii]]),
            np.array([P_ens[t, ii]]),
            K_ens[0,ii],
            Dt
        )

#   % Step 2: DA update
#   % In this step, we will update the model state using the observations as well 
#   % as model and observation uncertainties. First, we will add the model state in 
#   % the vector xMinus. We can then use this information to compute the empirical 
#   % error covariance matrix of the model state.
    
    # model prediction vector
    xMinus = S_tp1.copy()
    # empirical model error covariance matrix
    Cxx = (1 / (Ne - 1)) * (xMinus - np.mean(xMinus, axis=1, keepdims=True)) @ \
          (xMinus - np.mean(xMinus, axis=1, keepdims=True)).T
    

#   % Similarly, we will use the observation perturbations to compute the error 
#   % covariance matrix of the observations:

    # % empirical observation error covariance matrix
    Sll = (1 / (Ne - 1)) * dY[t, :] @ dY[t, :].T

#   % We now have sufficient information to fill the Kalman gain. 
#   % Please add the Kalman gain equation. You will have to use the 
#   % function np.linalg.inv to compute the inverse of matrices.

    # Kalman gain
    ############ TO BE FILLED ##################

#   % We can now apply the EnKF update to the model state stored in xMinus, 
#   % thus generating xPlus.

    # Kalman update
    xPlus = xMinus + K_gain[0,0] * (Y_ens[t, :] - (A @ xMinus).flatten())

#   % We can now initialize the next run with the updated information

    # Re-initialize
    S_t = xPlus

#   % Before we go forward, we will save all the update information in the preset 
#   % matrices, so that we can plot them after the process is done.

    # Save stats
    Cxpxp = (1 / (Ne - 1)) * (xPlus - np.mean(xPlus, axis=1, keepdims=True)) @ \
            (xPlus - np.mean(xPlus, axis=1, keepdims=True)).T

    idx = t - sim[0]
    xMinus_3d[:, :, idx] = xMinus
    xPlus_3d[:, :, idx] = xPlus
    Cxx_3d[:, :, idx] = Cxx
    Cxpxp_3d[:, :, idx] = Cxpxp
    Sll_2d[idx, 0] = Sll

## Visualize

Let us visualize the results.

In [ ]:
from functions.Visualization_functions import F_Visualize_Ex3

# %% Visualization
F_Visualize_Ex3.F_Visualize_Ex3(Y_ens, S_obs, S_true, S_ens, xPlus_3d, Sll_2d, Cxx_3d, Cxpxp_3d)

## Extra exercises

- Go to the [Marker for extra exercises] and triple the observation uncertainties (that is, triple the variable sigma_sObs). Then run the whole script again, and see what happens.
- Now, make the observation uncertainty 3 times smaller. Run the whole script again and see what happens. 

## Reflection questions

1. Do you think the observation and their uncertainty is realistic, given the ground truth? 

The ground truth lays within the ensemble of observations. Therefore, it can be considered a pretty realistic uncertainty (maybe slightly overestimated). 

2. What happens to the model estimates in the DA process? 

Model estimates pushed towards the observations and the truth. 

3. What happens to the model uncertainties in the DA process? 

The uncertainty levels are reduced. The uncertainty of DA is smaller than that of the OL and that of the observations. 

4. What do you think is the benefit of DA for models? 

Increased realism of the model while accounting for the uncertainties of both model and observations.
